In [1]:
import polars as pl
import numpy as np
import time
import ipaddress
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, balanced_accuracy_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
df_polars_raiz = pl.read_parquet('../dataset.parquet')

: 

In [3]:
df_polars = df_polars_raiz.sample(fraction=0.01, seed=42)

In [4]:
def ip_to_int(ip: str) -> int:
    try:
        return int(ipaddress.ip_address(ip))  # Funciona tanto para IPv4 quanto IPv6
    except ValueError:
        return None

In [5]:
#df_polars = df_polars.with_columns([
#    pl.col('id.resp_h').map_elements(ip_to_int).alias('id.resp_h'),
#    pl.col('id.orig_h').map_elements(ip_to_int).alias('id.orig_h')
#])

In [6]:
df_polars = df_polars.with_columns([
    pl.col('duration').fill_null(0),
    pl.col('orig_bytes').fill_null(0),
    pl.col('resp_bytes').fill_null(0)
])

In [7]:
lista_colunas = df_polars.columns
colunas_para_manter = ['ts', 'id.resp_p', 'history', 'conn_state', 'id.orig_p', 'orig_ip_bytes', 'label']  
colunas_para_dropar = [col for col in lista_colunas if col not in colunas_para_manter]
df_polars = df_polars.drop(colunas_para_dropar)

In [8]:
df_polars = df_polars.drop_nulls()


In [9]:
df_polars

ts,id.orig_p,proto,service,duration,conn_state,label
f64,i32,i64,i64,f64,i64,i32
1.5322e9,5526,0,0,0.0,0,1
1.5326e9,60403,1,0,0.0,2,1
1.5326e9,13386,1,0,0.0,2,1
1.5455e9,36097,0,0,0.0,0,1
1.5454e9,36097,0,0,0.0,0,1
…,…,…,…,…,…,…
1.5514e9,30535,0,1,0.000005,1,1
1.5454e9,36097,0,0,0.0,0,1
1.5514e9,41258,0,1,0.000002,1,1


In [10]:
df_polars = df_polars.with_columns(pl.col("ts").cast(pl.Int64))

In [11]:
df_polars

ts,id.orig_p,proto,service,duration,conn_state,label
i64,i32,i64,i64,f64,i64,i32
1532150893,5526,0,0,0.0,0,1
1532570324,60403,1,0,0.0,2,1
1532564882,13386,1,0,0.0,2,1
1545465243,36097,0,0,0.0,0,1
1545398682,36097,0,0,0.0,0,1
…,…,…,…,…,…,…
1551404209,30535,0,1,0.000005,1,1
1545414126,36097,0,0,0.0,0,1
1551402739,41258,0,1,0.000002,1,1


In [12]:
# Ordenar apenas pelo campo de tempo "ts"
df_polars = df_polars.sort("ts")

window_size = 5  # Tamanho da janela temporal
step_size = 1  # Passo entre janelas

def create_sequences(df, window_size, step_size):
    sequences, labels = [], []
    
    # Remover a coluna "ts" antes de converter para numpy
    group_np = df.drop("ts").to_numpy()
    
    for i in range(0, len(group_np) - window_size, step_size):
        seq = group_np[i:i + window_size, :-1]  # Características
        label = group_np[i + window_size - 1, -1]  # Última linha como rótulo
        sequences.append(seq)
        labels.append(label)
    
    return np.array(sequences), np.array(labels)


# Gerando X e y
X, y = create_sequences(df_polars, window_size, step_size)

In [13]:
print("Shape de X:", X.shape)
print("Shape de y:", y.shape)

Shape de X: (555228, 5, 5)
Shape de y: (555228,)


In [14]:
df_polars

ts,id.orig_p,proto,service,duration,conn_state,label
i64,i32,i64,i64,f64,i64,i32
1525879873,37334,0,0,2.999066,0,1
1525879921,40983,0,0,2.998808,0,0
1525879944,46566,0,0,0.0,0,1
1525879960,36497,0,0,0.0,0,1
1525880010,40761,0,0,0.0,0,1
…,…,…,…,…,…,…
1569018535,56399,1,0,0.0,4,1
1569018535,44790,1,0,0.0,4,1
1569018535,9799,1,0,0.0,4,1


In [15]:
scaler = MinMaxScaler()
X = X.reshape(-1, X.shape[-1])
X = scaler.fit_transform(X)
X = X.reshape(-1, window_size, X.shape[-1])
results = []

# Treinamento

In [16]:
class GRUClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_rate):
        super(GRUClassifier, self).__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, batch_first=True, dropout=dropout_rate, bidirectional=False)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        _, hidden = self.gru(x)
        hidden = self.relu(hidden[-1])
        hidden = self.dropout(hidden)
        return self.fc(hidden)

In [17]:
results = []

In [18]:
def startTrain(hidden_dim, dropout_rate):
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    start_training = time.time()
    results_fold = []
    num_epochs = 10  # Adicionado número de épocas para melhor treinamento

    for fold, (train_idx, test_idx) in enumerate(kfold.split(X, y), 1):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Normalização correta para evitar vazamento de dados
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(-1, window_size, X_train.shape[-1])
        X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(-1, window_size, X_test.shape[-1])

        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
        y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        input_dim = X_train.shape[2]
        #hidden_dim = 100
        #dropout_rate = 0.2
        output_dim = 1

        model = GRUClassifier(input_dim, hidden_dim, output_dim, dropout_rate).to(device)

        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        batch_size = 512
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size)

        for epoch in range(num_epochs):
            model.train()
            epoch_loss = 0
            for inputs, targets in train_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs.squeeze(), targets)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item() * inputs.size(0)
            print(f"Fold {fold}, Época {epoch+1}/{num_epochs}, Loss: {epoch_loss / len(train_loader.dataset):.4f}")
        
        end_training = time.time()

        model.eval()
        all_outputs, all_targets = [], []
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                all_outputs.append(outputs.cpu())
                all_targets.append(targets.cpu())

        all_outputs = torch.cat(all_outputs)
        all_targets = torch.cat(all_targets)
        
        y_pred = (all_outputs > 0.5).float().numpy()
        y_true = all_targets.numpy()

        confusion = confusion_matrix(y_true, y_pred)
        tn, fp, fn, tp = confusion.ravel()
        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        f1 = f1_score(y_true, y_pred)
        balanced_accuracy = balanced_accuracy_score(y_true, y_pred)
        false_alarm_rate = fp / (fp + tn) if (fp + tn) > 0 else 0
        
        training_duration = end_training - start_training
        evaluation_duration = time.time() - end_training
        avaliacao = [accuracy, balanced_accuracy, precision, recall, specificity, f1, false_alarm_rate, tn, fp, fn, tp, training_duration, evaluation_duration]
        print(avaliacao)
        results_fold.append(avaliacao)

    results_fold_array = np.array(results_fold, dtype=np.float32)
    mean_results = np.mean(results_fold_array, axis=0)
    results.append(["GRU"] + mean_results.tolist())

In [ ]:
for i in range(1,31):
    startTrain(hidden_dim=200,dropout_rate=0.2)
    print(i)

c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Fold 1, Época 1/10, Loss: 0.2767
Fold 1, Época 2/10, Loss: 0.1786
Fold 1, Época 3/10, Loss: 0.1438
Fold 1, Época 4/10, Loss: 0.1337
Fold 1, Época 5/10, Loss: 0.1293
Fold 1, Época 6/10, Loss: 0.1273
Fold 1, Época 7/10, Loss: 0.1252
Fold 1, Época 8/10, Loss: 0.1244
Fold 1, Época 9/10, Loss: 0.1239
Fold 1, Época 10/10, Loss: 0.1225
[0.9661671739639429, 0.902553816403628, 0.9644671632952031, 0.9963828005924908, 0.8087248322147651, 0.9801652456246865, 0.1912751677852349, 14460, 3420, 337, 92829, 278.4793612957001, 0.0]


c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Fold 2, Época 1/10, Loss: 0.2756
Fold 2, Época 2/10, Loss: 0.1745
Fold 2, Época 3/10, Loss: 0.1442
Fold 2, Época 4/10, Loss: 0.1344
Fold 2, Época 5/10, Loss: 0.1299
Fold 2, Época 6/10, Loss: 0.1275
Fold 2, Época 7/10, Loss: 0.1262
Fold 2, Época 8/10, Loss: 0.1253
Fold 2, Época 9/10, Loss: 0.1245
Fold 2, Época 10/10, Loss: 0.1232
[0.9665093744934532, 0.902818831375045, 0.964813219434728, 0.9964730601831007, 0.8091646025669894, 0.9803876071193144, 0.1908353974330106, 14374, 3390, 329, 92953, 545.4827289581299, 0.0]


c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


...

Fold 2, Época 1/10, Loss: 0.2747
Fold 2, Época 2/10, Loss: 0.1765
Fold 2, Época 3/10, Loss: 0.1424
Fold 2, Época 4/10, Loss: 0.1327
Fold 2, Época 5/10, Loss: 0.1281
Fold 2, Época 6/10, Loss: 0.1258
Fold 2, Época 7/10, Loss: 0.1248
Fold 2, Época 8/10, Loss: 0.1237
Fold 2, Época 9/10, Loss: 0.1234
Fold 2, Época 10/10, Loss: 0.1225
[0.9684815301766836, 0.9095982251105995, 0.9672738628083689, 0.996183615274115, 0.823012834947084, 0.9815159068824201, 0.176987165052916, 14620, 3144, 356, 92926, 772.7072694301605, 0.0]


c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Fold 3, Época 1/10, Loss: 0.2762
Fold 3, Época 2/10, Loss: 0.1753
Fold 3, Época 3/10, Loss: 0.1410
Fold 3, Época 4/10, Loss: 0.1324
Fold 3, Época 5/10, Loss: 0.1287
Fold 3, Época 6/10, Loss: 0.1268
Fold 3, Época 7/10, Loss: 0.1254
Fold 3, Época 8/10, Loss: 0.1247
Fold 3, Época 9/10, Loss: 0.1232
Fold 3, Época 10/10, Loss: 0.1225
[0.9673738810943213, 0.9059175827378199, 0.9657370435071957, 0.9964807244557462, 0.8153544410198935, 0.9808680407035999, 0.18464555898010648, 14550, 3295, 328, 92873, 1122.2704317569733, 0.0]


c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Fold 4, Época 1/10, Loss: 0.2789
Fold 4, Época 2/10, Loss: 0.1808
Fold 4, Época 3/10, Loss: 0.1443


In [53]:
metrics_df = pl.DataFrame(
    results,
    schema=['Algorithm', 'Accuracy', 'Balanced Accuracy', 'Precision', 'Recall', 'Specificity', 'F1-score', 'False Alarm Rate', 'tn', 'fp', 'fn', 'tp', 'training_duration', 'evaluation_duration']
)
metrics_df

c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\functools.py:888: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  return dispatch(args[0].__class__)(*args, **kw)


Algorithm,Accuracy,Balanced Accuracy,Precision,Recall,Specificity,F1-score,False Alarm Rate,tn,fp,fn,tp,training_duration,evaluation_duration
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""GRU""",0.957686,0.921282,0.965695,0.982009,0.860556,0.973755,0.139444,52190.199219,8455.200195,4355.200195,237743.796875,172.253525,0.0
"""GRU""",0.956743,0.918005,0.964073,0.982616,0.853393,0.973159,0.146607,51756.601562,8888.799805,4207.0,237892.0,181.652786,0.0
"""GRU""",0.952793,0.921479,0.967498,0.973738,0.869221,0.970536,0.130779,52713.0,7932.399902,6359.399902,235739.59375,203.197937,0.0


In [54]:
metrics_df.write_csv(f"metrics_results/unbalanced_GRU_metrics_output.csv", separator=';')